# Bandens python exercise 08

- Get the data from cars.csv file or make your own data
- Make a new table in your database with this data
- Make the following endpoints:

    - /api/car/{id} (GET)
    - /api/car/brand/{brand} (GET)
    - /api/car (POST)
    - /api/car/{id} (DELETE)


ved ikke hvordan det skulle aflevers da det er lavet med flask og derfor ligger uden for handin mappen men her er de filer jeg har lavet.

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import os

import mysql.connector as mysql

con_str = "mysql+mysqlconnector://root:root@db/db"
engine = create_engine(con_str)

class Cars_facade:
    def __init__(self, filename):
        print(os.getcwd() + filename)
        self.cnx = mysql.connect(host = "db", user = "root", passwd = "root", db = "db")
        self.data = pd.read_csv(filename, sep=';', skiprows=[1])
        self._index = None

    def replace(self):
        self.data = self.data.applymap(str)
        self.data.to_sql('cars',con=engine, if_exists='replace', index = True, index_label = 'id')
        return self.data
    
    def add_car( self, 
        car, 
        mpg, 
        cylinders, 
        displacement,
        horsepower,
        weight,
        acceleration,
        model,
        origin):
        id = self._next_index()
        self.cnx.connect()
        cur = self.cnx.cursor()
        query = "INSERT INTO cars values(%s,%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cur.execute(query, (id, car, mpg, cylinders, displacement, horsepower, weight, acceleration, model, origin))
        self.cnx.commit()
        cur.execute("SELECT * FROM cars where Car= %s AND MPG=%s AND Cylinders=%s", (car, mpg, cylinders))
        res = cur.fetchall()
        self.cnx.close()

        print(self._next_index())

        cur.close()

        return res 
        # {'Car': car, 'MPG': mpg, 'Cylinders': cylinders, 'Displacement': displacement, 'Horsepower': horsepower, 'Weight': weight, 'Acceleration': acceleration, 'Model': model, 'Origin': origin}

    def get_all_cars(self):
        self.cnx.connect()
        cur = self.cnx.cursor()
        cur.execute('SELECT * FROM cars')
        result = cur.fetchall()
        self.cnx.close()

        return result
    
    def get_car(self, id ):
        self.cnx.connect()
        cur = self.cnx.cursor()
        cur.execute('SELECT * from cars WHERE id=%s', (id, ))
        res = cur.fetchall()
        self.cnx.close()
        return res

    def delete_car(self, id ):
        res = self.get_car(id)
        self.cnx.connect()
        cur = self.cnx.cursor()
        cur.execute('DELETE from cars WHERE id=%s', (id, ))
        self.cnx.commit()
        self.cnx.close()
        return res

    def _next_index(self):
        if self._index:
            self._index += 1
        else:
            self.cnx.connect()
            cur = self.cnx.cursor()
            cur.execute('SELECT MAX(id) from cars')
            self._index = cur.fetchone()[0] + 1
            self.cnx.close()
        return self._index



def get_facade(filename):
    return Cars_facade(filename)

In [ ]:
#!flask/bin/python
import os, sys
currentdir = os.path.dirname(os.path.realpath(__file__))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from flask import Flask, jsonify, abort, request
from python_handin2.modules.Week8 import cars_facade

facade = cars_facade.get_facade('python_handin2/modules/Week8/cars.csv')
app = Flask(__name__)

@app.route('/api/car/<int:id>', methods=['GET'])
def get_car(id):
    return jsonify(facade.get_car(id))

@app.route('/api/car/<int:id>', methods=['DELETE'])
def delete_car(id):
    return jsonify(facade.delete_car(id))


@app.route('/api/car', methods=['POST'])
def add_car():
    return jsonify(facade.add_car(request.car, request.mpg, request.cylinders, request.displacement, request.horsepower, request.weight, request.acceleration, request.model, request.origin))


@app.route('/api/car/all', methods=['GET'])
def get_all_cars():
    return jsonify(facade.get_all_cars())

